In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\kidney-disease-classification'

In [14]:
#credentials

In [15]:
import tensorflow as tf

In [16]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    train_data_path: Path
    test_data_path: Path  # Added for test data path
    all_params: dict
    mlflow_tracking_uri: str
    params_image_size: list
    params_batch_size: int

In [18]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories,save_json

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        evaluation_config = ModelEvaluationConfig(
            model_path="artifacts/training/model.h5",
            train_data_path="artifacts/data_ingestion/kidney-ct-scan-image",
            test_data_path="artifacts/data_ingestion/kidney-ct-scan-image-test",  # NEW: test data path
            all_params=self.params,
            mlflow_tracking_uri="https://dagshub.com/arshpatel213/kidney-disease-classification.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return evaluation_config
    
    
    
    
    


In [20]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import numpy as np

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.2
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.train_data_path,
            subset="validation",
            shuffle=False,
            class_mode="binary",  # FIXED: Added missing class_mode
            **dataflow_kwargs
        )
        
    def train_valid_test_generator(self):
        datagenerator_kwargs = dict(rescale=1./255)
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Create test generator WITHOUT validation_split
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        # Reserve some data for testing (e.g., move 20% of each class to a separate folder)
        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.test_data_path,  # NEW: separate test folder
            shuffle=False,
            class_mode="binary",
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(model_path: str) -> tf.keras.Model:
        return tf.keras.models.load_model(model_path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self.train_valid_generator()
        
        # Add diagnostic information
        self.detailed_evaluation()
        
        self.results = self.model.evaluate(self.valid_generator)  # FIXED: Use self.model
        self.save_score()

    def detailed_evaluation(self):
        print(f"Validation samples: {self.valid_generator.samples}")
        print(f"Validation classes: {self.valid_generator.class_indices}")
        
        # Check class distribution
        labels = self.valid_generator.labels
        unique, counts = np.unique(labels, return_counts=True)
        print(f"Class distribution: {dict(zip(unique, counts))}")

    def save_score(self):
        scores={"loss": self.results[0], "accuracy": self.results[1]}
        save_json(path=Path("scores.json"), data=scores)
        print(f"Loss: {self.results[0]:.4f}, Accuracy: {self.results[1]:.4f}")
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_tracking_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.results[0], "accuracy": self.results[1]})
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="KidneyDiseaseModel")
            else:
                mlflow.keras.log_model(self.model, "model")
                


In [21]:
try:
    config=ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(eval_config)
    model_eval.evaluation()
    # model_eval.log_into_mlflow()
    
except Exception as e:
    print(f"Error occurred: {e}")
    raise e


YAML content loaded: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharin', 'local_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/updated_model.h5'}, 'training': {'root_dir': 'artifacts/training', 'model_checkpoint': 'artifacts/training/model.h5'}}
YAML content loaded: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'EPOCHS': 4, 'LEARNING_RATE': 0.001}
[2025-08-24 15:14:04,385] INFO in common: Created directory: artifacts
Found 93 images belonging to 2 classes.
Found 93 images belonging to 2 classes.
Validation samples: 93
Validation classes: {'Norma